In [49]:
from planet import api
from planet.api import filters
from sys import stdout
import urllib.request
import shutil
from requests.auth import HTTPBasicAuth
import os
import time
import requests
import cv2
import glob
import subprocess
import rasterio
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import wikipedia
import datetime

In [7]:
title = input("Please input your volcano of interest:")

Please input your volcano of interest: Anak Krakatau


In [10]:
page = wikipedia.WikipediaPage(title)

In [11]:
lat = float(page.coordinates[0])
long = float(page.coordinates[1])

In [12]:
start_date = input("Please input your start date (YYYY-MM-DD):")

Please input your start date (YYYY-MM-DD): 2018-11-12


In [13]:
end_date = input("Please input your end date (YYYY-MM-DD):")

Please input your end date (YYYY-MM-DD): 2018-11-21


In [14]:
api_key = input("Please input you Planet API key:") 

Please input you Planet API key: 219bc2e637384268a2d606dd8c05e7a7


In [15]:
client = api.ClientV1(api_key) 
PL_API_KEY = api_key

In [16]:
geo_json_geometry = {
  "type": "Polygon",
        "coordinates": [
          [
            [
              long - 0.07,
              lat + 0.07
            ],
            [
              long + 0.07,
              lat + 0.07
            ],
            [
              long + 0.07,
              lat - 0.07
            ],
            [
              long - 0.07,
              lat - 0.07
            ],
            [
              long - 0.07,
              lat + 0.07
            ]
          ]
        ]
      }

In [17]:
geometry_filter = {
    "type": "GeometryFilter",
    "field_name": "geometry",
    "config": geo_json_geometry
}

In [18]:
date_range_filter = {
    "type": "DateRangeFilter",
    "field_name": "acquired",
    "config": {
        "gte": start_date + "T00:00:00.000Z",
        "lte": end_date + "T00:00:00.000Z"
    }
}

In [19]:
cloud_cover_filter = {
    "type": "RangeFilter",
    "field_name": "cloud_cover",
    "config": {
        "lte": 0.2,
    }
}

In [20]:
item_types = ['PSScene4Band']
asset_type = 'analytic'

In [21]:
query = {
    "type": "AndFilter",
    "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [22]:
request = filters.build_search_request(
    query, item_types
)

In [23]:
dataset = client.quick_search(request)

stdout.write('id,cloud_cover,date\n')

id,cloud_cover,date


In [24]:
for item in dataset.items_iter(limit=None):
    props = item['properties']
    stdout.write('{0},{cloud_cover},{acquired}\n'.format(item['id'], **props))

20181119_025022_103c,0.02,2018-11-19T02:50:22.454971Z
20181117_024659_0e20,0.01,2018-11-17T02:46:59.204854Z
20181117_024701_0e20,0.02,2018-11-17T02:47:01.281035Z
20181117_024700_0e20,0.03,2018-11-17T02:47:00.242945Z
20181116_025055_1008,0,2018-11-16T02:50:55.681319Z
20181116_025054_1008,0.01,2018-11-16T02:50:54.635364Z
20181116_025053_1008,0,2018-11-16T02:50:53.588885Z
20181116_025052_1008,0,2018-11-16T02:50:52.542406Z
20181116_031618_1049,0,2018-11-16T03:16:18.753122Z
20181116_031617_1049,0,2018-11-16T03:16:17.773752Z
20181116_031619_1049,0,2018-11-16T03:16:19.733016Z


In [25]:
for i in item_types:
    request = api.filters.build_search_request(query, [i])  # ,name = None, interval= 'day')
    dataset = client.quick_search(request)
    print(i)
    for item in dataset.items_iter(limit=50):
        print(item['id'])
        session = requests.Session()
        session.auth = (PL_API_KEY, '')
        results = \
            session.get(
                ("https://api.planet.com/data/v1/item-types/" +
                 "{}/items/{}/assets/").format(i, item['id']))
       
        item_activation_url = results.json()[asset_type]["_links"]["activate"]
       
        response = session.post(item_activation_url)
        print(response.status_code)
        while response.status_code != 204:
            time.sleep(30)
            response = session.post(item_activation_url)
            response.status_code = response.status_code
            print(response.status_code)
        
        item_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(i, item['id'])
        result = requests.get(item_url, auth=HTTPBasicAuth(PL_API_KEY, ''))
        download_url = result.json()[asset_type]['location']  # KeyError: 'location'
        output_file = item['id'] + '.tif'

        with urllib.request.urlopen(download_url) as response, open(output_file, 'wb') as out_file:
            shutil.copyfileobj(response, out_file)

PSScene4Band
20181119_025022_103c
204
20181117_024659_0e20
204
20181117_024701_0e20
204
20181117_024700_0e20
204
20181116_025055_1008
204
20181116_025054_1008
204
20181116_025053_1008
204
20181116_025052_1008
204
20181116_031618_1049
204
20181116_031617_1049
204
20181116_031619_1049
204


In [26]:
os.getcwd()
os.mkdir('Original_Imagery')

In [27]:
sp = os.getcwd()
sf = os.listdir(sp)
dp = sp + ('\\Original_Imagery')
for file in sf:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sp,file), os.path.join(dp, file))

In [28]:
images = [f for f in os.listdir(dp) if os.path.isfile(os.path.join(dp, f))]

for image in images:
    folder_name = image.split('_')[0]

    new_path = os.path.join(dp, folder_name)
    if not os.path.exists(new_path):
        os.makedirs(new_path)

    old_image_path = os.path.join(dp, image)
    new_image_path = os.path.join(new_path, image)
    shutil.move(old_image_path, new_image_path)

In [29]:
folders = []
for r, d, f in os.walk(dp):
    for folder in d:
        folders.append(os.path.join(r, folder))

i = 0
for x in folders:
    files_to_mosaic = glob.glob(folders[i] + '/*.tif')
    files_string = " ".join(files_to_mosaic)
    command = "python " + sp + "\gdal_merge.py -o "+ files_string[-33:-25] + ".tif" + " -of gtiff " + files_string
    print(command)
    i = i + 1
    output = subprocess.getoutput(command)
    print(output)

python C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\gdal_merge.py -o 20181116.tif -of gtiff C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\Original_Imagery\20181116\20181116_025052_1008.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\Original_Imagery\20181116\20181116_025053_1008.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\Original_Imagery\20181116\20181116_025054_1008.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\Original_Imagery\20181116\20181116_025055_1008.tif C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\Original_Imagery\20181116\20181116_031617_1049.tif C:\Users\maddi\Downloads\PlanetC

In [30]:
os.mkdir('Merged_Tifs')

In [31]:
sp2 = os.getcwd()
sf2 = os.listdir(sp2)
dp2 = sp2 + ('\\Merged_Tifs')
for file in sf2:
    if file.endswith('.tif'):
        shutil.move(os.path.join(sp2,file), os.path.join(dp2, file))

In [32]:
os.chdir(sp2 + '\\Merged_Tifs')

In [78]:
files = []
for r, d, f in os.walk(dp2):
    for file in f:
        files.append(os.path.join(r, file))
timeavg = []       
ndviavg = []
i = 0
for x in files:
    with rasterio.open(files[i]) as src:
            band_red = src.read(3)
    with rasterio.open(files[i]) as src:
            band_nir = src.read(4)
            
    np.seterr(divide='ignore', invalid='ignore')
    ndvi = ((band_nir.astype(float) - band_red.astype(float)) / (band_nir + band_red))
    
    
    kwargs = src.meta
    kwargs.update(
    dtype = rasterio.float32,
    count = 1)
    with rasterio.open('ndvi_' + f[i], 'w', **kwargs) as dst:
        dst.write_band(1, ndvi.astype(rasterio.float32))
        
    ndvi2 = np.ma.masked_where(ndvi < 0., ndvi)
    
    fig2 = plt.figure(figsize=(10,10))
    ax = fig2.add_subplot(111)
    
    plt.title(f[i] + " NDVI Histogram", fontsize=18, fontweight='bold')
    plt.xlabel("NDVI values", fontsize=14)
    plt.ylabel("# pixels", fontsize=14)
    
    b = ndvi2[~np.isnan(ndvi2)]
    numBins = 20
    ax.hist(b,numBins,color='blue',alpha=0.8)
    
    fig2.savefig("ndvi_" + f[i][0:8] + "_histogram.png", dpi=200, bbox_inches='tight', pad_inches=0.7)

    
    ndvi_adjusted = ndvi * (-255.0)
    masked_ndvi_adjusted = np.ma.array(ndvi_adjusted, mask=np.isnan(ndvi_adjusted))
    cmap = plt.cm.RdYlGn
    cmap.set_bad(color = 'white',alpha = 0.0)
    img = masked_ndvi_adjusted.astype(np.uint8)
    plt.imsave("ndvi_" + f[i][0:8] + "_colormap.tiff", img, cmap=cmap)
    plt.close()
    
    avg = np.average(b)
    ndviavg.append(avg)
    time = f[i][0:8]
    timeavg.append(time)
 
    i = i + 1
    
fig3 = plt.figure(figsize=(10,10))
ax2 = fig3.add_subplot(111)
ax2.set_ylim([0, 1])
plt.plot(timeavg, ndviavg, color = 'blue', linewidth = 4)
plt.axhline(y=0.3, color='r', linestyle='-', linewidth = 3, label = 'Below: Unhealthy')
plt.axhline(y=0.6, color='y', linestyle='-', linewidth = 3, label = 'Below: Somewhat Healthy')
plt.axhline(y=0.7, color='g', linestyle='-', linewidth = 3, label = 'Above: Healthy')
plt.legend()
plt.title("Average NDVI Over Time", fontsize=18, fontweight='bold')
plt.xlabel("Start Date & End Date", fontsize=14)
plt.ylabel("Average NDVI", fontsize=14)
fig3.savefig("AverageNDVI_OverTime.png", dpi=200, bbox_inches='tight', pad_inches=0.7)
plt.close()

In [79]:
sp3 = os.getcwd()
sf3 = os.listdir(sp3)

for file in sf3:
    if file.startswith('ndvi'):
        shutil.move(os.path.join(sp3,file), os.path.join(sp2,file))

In [80]:
os.chdir(sp2)
os.mkdir('Vegetation_Indices')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'Vegetation_Indices'

In [81]:
sp4 = os.getcwd()
sf4 = os.listdir(sp4)
dp4 = sp2 + '\\Vegetation_Indices'

for file in sf3:
    if file.startswith('ndvi'):
        shutil.move(os.path.join(sp4,file), os.path.join(dp4,file))

In [82]:
sp3 = os.getcwd()
sf3 = os.listdir(sp3)

for file in sf3:
    if file.startswith('Ave'):
        shutil.move(os.path.join(sp3,file), os.path.join(sp2,file))

In [83]:
os.chdir(sp2)
os.mkdir('Average_NDVI')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'Average_NDVI'

In [84]:
sp4 = os.getcwd()
sf4 = os.listdir(sp4)
dp4 = sp2 + '\\Average_NDVI'

for file in sf3:
    if file.startswith('Ave'):
        shutil.move(os.path.join(sp4,file), os.path.join(dp4,file))

Error: Cannot move a directory 'C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\Average_NDVI' into itself 'C:\Users\maddi\Downloads\PlanetCode-Integration-and-sorting-patch-2\PlanetCode-Integration-and-sorting-patch-2\Average_NDVI\Average_NDVI'.